#Langchain PDF Q/A RAG System:

In [1]:
!pip install langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 8.0 MB/s eta 0:00:00


Here I uploaded a book "Code Complete by Steve McConnell" as a pdf document

In [4]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader("/content/Steve McConnell - Code Complete (2nd edition).pdf")

pages = pdf_loader.load_and_split()

In [5]:
pages

[Document(metadata={'source': '/content/Steve McConnell - Code Complete (2nd edition).pdf', 'page': 1}, page_content='PUBLISHED BY\nMicrosoft Press\nA Division of Microsoft Corporation\nOne Microsoft Way\nRedmond, Washington 98052-6399\nCopyright © 2004 by Steven C. McConnell\nAll rights reserved. No part of the contents of this book may be reproduced or transmitted in any form or by \nany means without the written permission of the publisher.\nLibrary of Congress Cataloging-in-Publication Data\nMcConnell, Steve\nCode Complete / Steve McConnell.--2nd ed.\np.  cm.\nIncludes index.\nISBN 0-7356-1967-0\n1. Computer Software--Development--Handbooks, manuals, etc. I. Title.\nQA76.76.D47M39 2004\n005.1--dc22 2004049981\nPrinted and bound in the United States of America.\n15 16 17 18 19 20 21 22 23 24 QGT 6 5 4 3 2 1\nDistributed in Canada by H.B. Fenn and Company Ltd. A CIP catalogue record for this book is available from \nthe British Library.\nMicrosoft Press books are available through bo

#Splitter:

In [6]:
!pip install -qU langchain-text-splitters

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

txt_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 20,
    length_function = len,
    is_separator_regex=False
)

In [8]:
chunk_list = []
for page in pages:
  chunks = txt_splitter.split_text(page.page_content)
  for chunk in chunks:
    chunk_list.append(chunk)
  print(len(chunk_list))

17
18
32
45
46
63
78
104
134
166
183
212
239
267
294
304
318
334
347
364
379
399
406
423
427
432
433
443
445
458
474
488
502
514
517
525
536
547
560
578
586
597
617
637
655
672
689
704
720
732
745
764
781
786
796
813
831
851
867
882
897
908
919
930
944
958
965
981
994
1006
1020
1036
1056
1070
1083
1096
1115
1132
1148
1166
1183
1201
1217
1237
1256
1269
1283
1301
1314
1332
1345
1350
1362
1378
1394
1409
1423
1441
1462
1480
1495
1507
1511
1523
1540
1549
1566
1583
1601
1618
1634
1649
1657
1667
1680
1697
1717
1734
1748
1766
1779
1792
1809
1819
1836
1855
1875
1893
1913
1932
1951
1969
1985
2004
2021
2038
2055
2072
2080
2098
2114
2131
2148
2164
2183
2198
2211
2231
2248
2266
2281
2300
2311
2318
2330
2347
2360
2378
2390
2405
2420
2435
2449
2459
2472
2487
2507
2522
2539
2548
2566
2583
2599
2617
2632
2652
2667
2678
2696
2715
2730
2749
2768
2786
2800
2813
2828
2837
2849
2853
2865
2878
2897
2912
2932
2946
2953
2972
2989
3010
3028
3047
3063
3082
3091
3110
3125
3141
3157
3172
3189
3204
3216
3229
3242
3

In [9]:
chunk_list[100]

'4.1 Choice of Programming Language . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 61'

#Metadata Preprocessing:

In [10]:
from langchain.docstore.document import Document

In [11]:
doc_list = []

for page in pages:
  pag_split = txt_splitter.split_text(page.page_content)

  for pag_sub_split in pag_split:
    metadata = {"source": "Book", "page_no": page.metadata.get('page', 0) + 1}
    doc_string = Document(page_content= pag_sub_split, metadata=metadata)
    doc_list.append(doc_string)


In [12]:
doc_list[26]

Document(metadata={'source': 'Book', 'page_no': 4}, page_content='Subtitled ‘A Practical Handbook of Software Construction,’ this 850-page book is exactly \nthat. Its stated goal is to narrow the gap between the knowledge of ‘industry gurus and pro-')

In [13]:
len(doc_list)

13518

#Embeddings:

In [14]:
!pip install sentence_transformers langchain_huggingface

In [15]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

In [16]:
model_name = 'BAAI/bge-small-en'
embed_model = HuggingFaceBgeEmbeddings(model_name=model_name)

<ipython-input-16-7e779377dc83>:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceBgeEmbeddings(model_name=model_name)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#**Qdrant Vectore Store:**

##Qdrant Credentials:

In [17]:
!pip install qdrant_client langchain-qdrant

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 26.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.68.1
    Uninstalling grpcio-1.68.1:
      Successfully uninstalled grpcio-1.68.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but y

In [ ]:
qdrant_url="https://d0413-ae37-8001e4736ed5.europe-west3-0.gcp.3541"
qdrant_key="74qTUBZqABCj-JLgrV_U2654FXQ-Rg"
groq_api_key="gsk_qTkwehjf"

collection_name = 'Steve_McConnel_code_book'

In [ ]:
from qdrant_client import qdrant_client
from langchain_qdrant import QdrantVectorStore

qdrant = QdrantVectorStore.from_documents(
    doc_list,
    embed_model,
    url=qdrant_url,
    api_key=qdrant_key,
    collection_name=collection_name
)

  # Q/A Chain

In [ ]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 10.4 MB/s eta 0:00:00


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

Retrievers:

In [ ]:
retriever = qdrant.as_retriever()

In [ ]:
retriever_results = retriever.invoke("What is a good sofware architecture?")
print(retriever_results)


[Document(metadata={'source': 'Book', 'page_no': 84, '_id': 'd816206c-000e-403b-8e87-b91910834cbc', '_collection_name': 'Steve_McConnel_code_book'}, page_content='ified in the software architecture. The architecture should specify major elements of \nWeb page formats, GUIs, command line interfaces, and so on. Careful architecture of'), Document(metadata={'source': 'Book', 'page_no': 84, '_id': '63d6e7e1-7c43-4822-b342-5a2708e936a3', '_collection_name': 'Steve_McConnel_code_book'}, page_content='ified in the software architecture. The architecture should specify major elements of \nWeb page formats, GUIs, command line interfaces, and so on. Careful architecture of'), Document(metadata={'source': 'Book', 'page_no': 84, '_id': '827efe7e-d1ca-4128-93a9-ad1f9f30963a', '_collection_name': 'Steve_McConnel_code_book'}, page_content='ified in the software architecture. The architecture should specify major elements of \nWeb page formats, GUIs, command line interfaces, and so on. Careful archite

In [ ]:

prompt_str = """Answer the question based only on the following context:
{context}
Question : {question}
Answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_str)

def doc2str(docs):
  return '\n\n'.join(doc.page_content for doc in docs)

llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0, groq_api_key=groq_api_key)

rag_chain = (
    {
        'context': retriever | doc2str, "question": RunnablePassthrough()
        }
    | prompt | llm | StrOutputParser()
)

In [ ]:
query ="What is a good sofware architecture?"

response = rag_chain.invoke(query)
print(response)

A good software architecture is one that carefully specifies major elements of Web page formats, graphical user interfaces (GUIs), command line interfaces, and so on. It provides a clear structure for the software system, enabling effective organization, development, and maintenance of the software. A well-designed architecture ensures that the system's components interact efficiently, are modular, reusable, and adaptable to changing requirements.
